# Copula Fit

Estimate rho_event and rho_size from anonymized data using rank-normalized correlations.

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
import os
from pathlib import Path
RUN_TAG = os.environ.get("RUN_TAG")
HIST_END = os.environ.get("HIST_END")
if RUN_TAG is None:
    RUN_TAG = HIST_END or "2025Q3"
BASE_OUT = Path("model_fits") / "runs" / RUN_TAG
CALIB_DIR = BASE_OUT / "calibration"

INPUT_PATH = "anonymized.csv"
OUT_PATH = str(CALIB_DIR / "copula_params.json")

if not Path(INPUT_PATH).exists():
    candidates = list(Path.cwd().glob("**/anonymized.csv"))
    if not candidates:
        candidates = list(Path.cwd().parent.glob("**/anonymized.csv"))
    if not candidates:
        candidates = list(Path.cwd().parent.parent.glob("**/anonymized.csv"))
    if candidates:
        INPUT_PATH = str(candidates[0])
    else:
        raise FileNotFoundError("anonymized.csv not found. Set INPUT_PATH to the full path.")

print("Using INPUT_PATH:", INPUT_PATH)
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)


Using INPUT_PATH: anonymized.csv


In [3]:
def _norm_key(s: str) -> str:
    return " ".join(s.strip().lower().replace("_", " ").split())


def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    col_map = {_norm_key(c): c for c in df.columns}
    def _get(name: str) -> str:
        k = _norm_key(name)
        return col_map.get(k, name)

    rename = {}
    rename[_get("Adj strategy")] = "Adj Strategy"
    rename[_get("Adj Strategy")] = "Adj Strategy"
    rename[_get("Quarter of Transaction Date")] = "Quarter"
    rename[_get("Year of Transaction Date")] = "Year"
    rename[_get("FundID")] = "FundID"
    rename[_get("Adj Drawdown EUR")] = "Adj Drawdown EUR"
    rename[_get("Adj Repayment EUR")] = "Adj Repayment EUR"
    rename[_get("Recallable")] = "Recallable"
    rename[_get("Capacity")] = "Capacity"
    rename[_get("NAV Adjusted EUR")] = "NAV Adjusted EUR"
    return df.rename(columns=rename)


def parse_quarter(q) -> float:
    if pd.isna(q):
        return np.nan
    if isinstance(q, (int, np.integer, float, np.floating)):
        return float(q)
    s = str(q).strip().upper()
    if s.startswith("Q"):
        s = s[1:]
    try:
        return float(s)
    except Exception:
        return np.nan


def add_quarter_end(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["Quarter"] = df["Quarter"].apply(parse_quarter)
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    m = df["Year"].notna() & df["Quarter"].notna()
    years = df.loc[m, "Year"].astype(int)
    quarters = df.loc[m, "Quarter"].astype(int)
    df.loc[m, "quarter_end"] = pd.PeriodIndex(year=years, quarter=quarters, freq="Q").to_timestamp("Q")
    return df


def rank_to_normal(x: pd.Series) -> pd.Series:
    x = x.copy()
    n = x.notna().sum()
    if n < 3:
        return pd.Series(index=x.index, dtype=float)
    ranks = x.rank(method="average", na_option="keep")
    u = (ranks - 0.5) / n
    u = u.clip(1e-6, 1 - 1e-6)
    return pd.Series(norm.ppf(u), index=x.index)


def avg_offdiag_corr(mat: pd.DataFrame, min_obs: int = 5) -> float:
    if mat.shape[1] < 2:
        return float("nan")
    # drop columns with too few obs or zero variance
    keep = []
    for c in mat.columns:
        s = mat[c].dropna()
        if len(s) < min_obs:
            continue
        if s.nunique() <= 1:
            continue
        keep.append(c)
    if len(keep) < 2:
        return float("nan")
    corr = mat[keep].corr()
    vals = corr.values
    n = vals.shape[0]
    if n <= 1:
        return float("nan")
    return float((vals.sum() - np.trace(vals)) / (n * (n - 1)))
    corr = mat.corr()
    vals = corr.values
    n = vals.shape[0]
    if n <= 1:
        return float("nan")
    return float((vals.sum() - np.trace(vals)) / (n * (n - 1)))

In [4]:
# --- Load + compute ---

df = pd.read_csv(INPUT_PATH, engine="python")
df = normalize_columns(df)
df = add_quarter_end(df)

# --- Optional historical cutoff ---
HIST_END_QE = None
if HIST_END:
    try:
        y = int(HIST_END[:4])
        q = int(HIST_END[-1])
        HIST_END_QE = pd.Period(f"{y}Q{q}", freq="Q").to_timestamp("Q")
    except Exception:
        HIST_END_QE = None
if HIST_END_QE is not None:
    df = df[df["quarter_end"] <= HIST_END_QE].copy()
    print(f"Filtered history to <= {HIST_END_QE.date()} (rows={len(df)})")
df = df.dropna(subset=["FundID", "quarter_end"])
df = df.sort_values(["FundID", "quarter_end"])

# Event indicators

df["draw_event"] = pd.to_numeric(df["Adj Drawdown EUR"], errors="coerce").fillna(0.0) > 0

df["rep_event"] = pd.to_numeric(df["Adj Repayment EUR"], errors="coerce").fillna(0.0) > 0

# Ratios for size

df["nav_prev"] = df.groupby("FundID")["NAV Adjusted EUR"].shift(1)
cap = pd.to_numeric(df["Capacity"], errors="coerce")
draw = pd.to_numeric(df["Adj Drawdown EUR"], errors="coerce")
rep = pd.to_numeric(df["Adj Repayment EUR"], errors="coerce")
nav_prev = pd.to_numeric(df["nav_prev"], errors="coerce")


df["draw_ratio"] = np.where(cap > 0, draw / cap, 0.0)

df["rep_ratio"] = np.where(nav_prev.abs() > 1.0, rep / nav_prev.abs(), 0.0)

# Wide matrices

draw_event_w = df.pivot_table(index="quarter_end", columns="FundID", values="draw_event", aggfunc="last")
rep_event_w = df.pivot_table(index="quarter_end", columns="FundID", values="rep_event", aggfunc="last")

draw_ratio_w = df.pivot_table(index="quarter_end", columns="FundID", values="draw_ratio", aggfunc="last").fillna(0.0)

rep_ratio_w = df.pivot_table(index="quarter_end", columns="FundID", values="rep_ratio", aggfunc="last").fillna(0.0)

# Rank-normalize per fund

draw_event_z = draw_event_w.apply(rank_to_normal, axis=0)
rep_event_z = rep_event_w.apply(rank_to_normal, axis=0)

draw_ratio_z = draw_ratio_w.apply(rank_to_normal, axis=0)
rep_ratio_z = rep_ratio_w.apply(rank_to_normal, axis=0)

avg_corr_draw_event = avg_offdiag_corr(draw_event_z)
avg_corr_rep_event = avg_offdiag_corr(rep_event_z)
avg_corr_draw_size = avg_offdiag_corr(draw_ratio_z)
avg_corr_rep_size = avg_offdiag_corr(rep_ratio_z)


# Replace NaN correlations with 0.0 for stability
avg_corr_draw_event = 0.0 if not np.isfinite(avg_corr_draw_event) else avg_corr_draw_event
avg_corr_rep_event = 0.0 if not np.isfinite(avg_corr_rep_event) else avg_corr_rep_event
avg_corr_draw_size = 0.0 if not np.isfinite(avg_corr_draw_size) else avg_corr_draw_size
avg_corr_rep_size = 0.0 if not np.isfinite(avg_corr_rep_size) else avg_corr_rep_size
# Suggested rho: corr ≈ rho^2

def rho_from_corr(c):
    if not np.isfinite(c) or c <= 0:
        return 0.0
    return float(np.sqrt(min(c, 0.99)))

rho_event = rho_from_corr(np.nanmean([avg_corr_draw_event, avg_corr_rep_event]))
rho_size = rho_from_corr(np.nanmean([avg_corr_draw_size, avg_corr_rep_size]))

out = {
    "avg_corr_draw_event": avg_corr_draw_event,
    "avg_corr_rep_event": avg_corr_rep_event,
    "avg_corr_draw_size": avg_corr_draw_size,
    "avg_corr_rep_size": avg_corr_rep_size,
    "rho_event": rho_event,
    "rho_size": rho_size,
}

with open(OUT_PATH, "w") as f:
    json.dump(out, f, indent=2)

print("Wrote:", OUT_PATH)
print(out)


/var/folders/w3/y0xpfndx1099qx0947v35sxh0000gn/T/ipykernel_19020/2151741316.py:46: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  df.loc[m, "quarter_end"] = pd.PeriodIndex(year=years, quarter=quarters, freq="Q").to_timestamp("Q")


Filtered history to <= 2025-09-30 (rows=30857)


Wrote: model_fits/runs/2025Q3/calibration/copula_params.json
{'avg_corr_draw_event': 0.0, 'avg_corr_rep_event': 0.0, 'avg_corr_draw_size': 0.16190288878026496, 'avg_corr_rep_size': 0.12665409041848738, 'rho_event': 0.0, 'rho_size': 0.3798400842451678}
